In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import StandardScaler

In [3]:
#sigmoid fonk f(x)=1/(1+exp^-x)
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [4]:
def deriv_sigmoid(x):
    return(x*(1-x))

In [5]:
def relu(x):
    return np.maximum(0,x)

In [6]:
def deriv_relu(x):
    der=list()
    der.clear()
    for val in x:
        if val>0:
            der.append(1)
        else:
            der.append(0)
    return der

In [7]:
def leaky_relu(x):
    return np.maximum(0.1*x,x)
    

In [8]:
def deriv_leaky_relu(x):
    der=list()
    der.clear()
    for val in x:
        if val>0:
            der.append(1)
        else:
            der.append(0.1)
    return der

In [9]:
def initialize(x, neuron_num):
    np.random.seed(12345)
    w1=2*np.random.random((len(x.columns), neuron_num))-1
    w2=2*np.random.random((neuron_num, 1))-1
    bias1=2*np.random.random()-1
    bias2=2*np.random.random()-1
    return w1, w2, bias1, bias2

In [10]:
def forward_prop(w1, w2, bias1, bias2, x):
    x=x.reshape(13,1)
    z1=np.dot(w1.T,x)+bias1
    out1=relu(z1)
    #out1=leaky_relu(z1)
    z2=np.dot(w2.T,out1)+bias2
    out2=relu(z2) 
    #out2=leaky_relu(z2)
    return z1, out1, z2, out2

In [11]:
def back_prop(w1, w2, bias1, bias2, x, y, z1, out1, z2, out2, lr, j):
    x=x.reshape(13,1)
    err_2=2*(np.subtract(out2.sum(axis=0), y)) # karenin turevi oldugu icin 2*(y_pred-y_act)
    dummy_2=np.array(deriv_relu(z2)).reshape(1,1)
    #dummy_2=np.array(deriv_leaky_relu(z2)).reshape(1,1)
    dc_dw2=lr*np.dot(out1,dummy_2)*err_2
    w2=np.subtract(w2,dc_dw2)
    dc_db2=lr*err_2
    bias2-=dc_db2

    err_1= 2*(np.subtract(out1.sum(axis=0), y))
    dummy_1=np.array(deriv_relu(z1)).reshape(10,1)
    #dummy_1=np.array(deriv_leaky_relu(z1)).reshape(7,1)
    dc_dw1=lr*np.dot(x,dummy_1.T)*err_1
    w1=np.subtract(w1,dc_dw1)
    dc_db1=lr*err_1
    bias1-=dc_db1
    return w1, w2, bias1, bias2

In [12]:
def forward_prop_test(w1, w2, bias1, bias2, x):
    x=x.reshape(13,1)
    z1=np.dot(w1.T,x)+bias1
    out1=relu(z1)
    z2=np.dot(w2.T,out1)+bias2
    out2=relu(z2) 
    return out1

In [13]:
pd.set_option("display.max_rows", None,
             "display.max_columns", None)

In [14]:
columns=("CRIM","ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV")

In [15]:
ds=pd.read_csv("/Users/ecemdenizbabaoglan/Desktop/TOBBETU/yap470/housing.csv", sep='\s+', engine='python', names=columns)

In [16]:
#ds.head()

In [17]:
#ds.isnull().sum()

In [18]:
#ds.describe(include='all')

In [19]:
target=ds.loc[:,'MEDV'].copy()

In [20]:
data=ds.drop('MEDV', axis=1).copy()

In [21]:
w1, w2, bias1, bias2=initialize(data,10)

In [22]:
scaler=StandardScaler()
data_normalized=scaler.fit_transform(data)
target=target.to_numpy()

In [23]:
cutoff = int(len(data) * 0.8)
x_train, x_test = data_normalized[:cutoff], data_normalized[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [24]:
epochs=100

In [25]:
lr=0.000001

In [26]:
output1=list()
output2=list()
z_1=list()
z_2=list()
loss=0
loss_test=0

In [27]:
for e in range(epochs):
    output1.clear()
    output2.clear()
    z_1.clear()
    z_2.clear()
    errors=0
    for i in range(len(y_train)):   
        z1, out1, z2, out2= forward_prop(w1, w2, bias1, bias2, x_train[i])
        output1.append(out1)
        output2.append(out2)
       # predicted[i]=output[i].sum(axis=0)
        w1, w2, bias1, bias2 = back_prop(w1, w2, bias1, bias2, x_train[i], y_train[i], z1, output1[i], z2, output2[i], lr, i+1)

In [28]:
for i in range(len(y_train)):
   # print(y_train[i], output[i].sum(axis=0))
    loss+=(y_train[i]-(output2[i].sum(axis=0)))

In [29]:
print(loss/(len(y_train)))

[3.5739411]


In [30]:
out_test=list()
for i in range(len(y_test)): 
    #z_test1, out_test1, z_test2, out_test2=forward_prop(w1, w2, bias1, bias2, x_test[i])
    #out_test.append(out_test2)
    out_test.append(forward_prop_test(w1, w2, bias1, bias2, x_test[i]))
    loss_test+=(y_test[i]-(out_test[i].sum(axis=0)))
print(loss_test/(len(y_test)))

[7.13917587]


In [31]:
for i in range(len(y_test)):
    print(y_test[i], out_test[i].sum(axis=0))

8.5 [16.54852391]
5.0 [24.71442109]
11.9 [14.18593666]
27.9 [10.75967668]
17.2 [9.30452684]
27.5 [7.98799634]
15.0 [11.1593323]
17.2 [4.59671078]
17.9 [9.85913489]
16.3 [10.93895557]
7.0 [15.08057516]
7.2 [5.6621409]
7.5 [3.6424596]
10.4 [9.41973359]
8.8 [16.83470801]
8.4 [3.89907292]
16.7 [10.78462567]
14.2 [9.38290911]
20.8 [8.94130827]
13.4 [3.46218851]
11.7 [3.31191924]
8.3 [5.01427431]
10.2 [2.98402294]
10.9 [7.84762922]
11.0 [4.10990232]
9.5 [4.49682752]
14.5 [3.61535102]
14.1 [4.17401685]
16.1 [3.17904979]
14.3 [4.09945522]
11.7 [5.95628899]
13.4 [5.84586617]
9.6 [5.1083384]
8.7 [5.16725226]
8.4 [7.16767844]
12.8 [11.89890547]
10.5 [14.46331163]
17.1 [12.14671865]
18.4 [12.32389066]
15.4 [12.70059601]
10.8 [9.18069754]
11.8 [4.59868154]
14.9 [9.58095249]
12.6 [12.26829875]
14.1 [12.05017004]
13.0 [9.57034478]
13.4 [3.91081374]
15.2 [10.88045674]
16.1 [10.79077909]
17.8 [13.42032209]
14.9 [4.10389607]
14.1 [3.76382202]
12.7 [3.09186886]
13.5 [3.40160875]
14.9 [7.42623346]
20.0 [1

In [32]:
#for i in range(len(y_train)):
#    print(y_train[i], output2[i])

In [33]:
out_test_=list()
loss_test_=list()
for i in range(len(y_train)): 
    #z_test1, out_test1, z_test2, out_test2=forward_prop(w1, w2, bias1, bias2, x_test[i])
    #out_test.append(out_test2)
    out_test_.append(forward_prop_test(w1, w2, bias1, bias2, x_train[i]))
    loss_test_+=(y_train[i]-(out_test_[i].sum(axis=0)))
print(loss_test_/(len(y_train)))

[]
